In [1]:
import pandapower as pp
import pandas as pd
import numpy as np
from numpy.linalg import inv
from pp_heig_simulation import load_net_from_xlsx

# Exercise power flow

In this exercise, we will dive into the power flow analysis of a 5-mesh electrical network topology (figure below).

The objective is to understand and implement a complete robust power flow algorithm, bringing clarity to complex electricity grid calculations. Refer to the figure below for a visual representation of a 5-mesh network.

Checklist of the exercise:

1. `cartesian2z` and `polar2z`
    - Transform coordinates into complex numbers.
2. `convert_line_data_in_pu`: 
    - Fill line_pu DataFrame missing columns.
3. `compute_admittance_matrix`
    - Calculate admittance matrix elements.
4. `update_bus_power`: 
    - Decompose complex voltages and admittance matrix.
    - Update bus powers.
5. `jacobian_matrix`: 
    - Decompose complex voltages and admittance matrix.
    - Calculate active and reactive power partial differential depending on voltages modulus and angle.
6. `update_voltage`:     
    - Calculate state variable delta.
    - Update voltages polars coordinates.
    - Build complex voltages.
7. `process_results`:     
    - For each line find parameters: from_bus, to_bus, longitudinal admittance and transversal susceptance.
    - Calculate line current in pu.
    - Calculate line apparent power in pu.
8. Power flow algorithm
    - Define grid base values.
    - Define slack bus index.
    - Implement algorithm steps 5 to 8
  
You'll have to go through all the functions and code the part we're asking you to do.

<img alt="5-mesh power network" width="600" caption="5-mesh power network" src="./5-mesh_electrical_power_grid.png" id="5-mesh_grid"/>
<a href="./5-mesh_electrical_power_grid.png"></a>

*Figure – 5-mesh electrical power grid*

## Used functions
### Remove slack bus

In [2]:
def remove_slack_bus(data_np: np.array, slack_bus: int) -> np.array:
    """Remove data associated with the slack bus from a 1 or 2-dimensional NumPy array.
    
    Parameters
    ----------
    data_np : np.array 
        The NumPy array of the data to be processed.
    slack_bus : int
        The index of the slack bus to be removed.

    Returns
    -------
    numpy.ndarray
        A new NumPy array with the data related to the slack bus removed

    Raises
    ------
    ValueError
        If `data_np` is not a NumPy array, or if the `slack_bus` index is out of bounds.

    Notes
    -----
    This function removes data associated with the slack bus, specified by its index, from a 1 or 2-dimensional NumPy array.
    """
    if len(data_np.shape) == 2:
        return np.delete(np.delete(data_np, slack_bus, axis=0), slack_bus, axis=1)
    elif data_np.ndim == 1:
        return np.delete(data_np, slack_bus)
    else:
        return None

### Convert cartesian and polar coordinates to complex numbers

For polar coordinates, please refer to RHT RE04-5 course, slide 27.

In [ ]:
def cartesian2z(real: np.array, imag: np.array) -> np.array:
    """Convert cartesian coordinates to a complex number.

    Parameters
    ----------
    real : float
        An array of complex numbers' real part.
    imag : float
        An array of complex numbers' imaginary part.

    Returns
    -------
    complex
        A complex number represented in rectangular coordinates.
    """
    return ##YOUR CODE HERE##

In [3]:
def polar2z(module: np.array, angle: np.array) -> np.array:
    """Convert polar coordinates to a complex number.

    Parameters
    ----------
    module : np.array
        An array of complex numbers' magnitude (module).
    angle : np.array
        An array of complex numbers' angle (in radians).

    Returns
    -------
    np.array
        An array of complex number
    """
    return ##YOUR CODE HERE##

### Convert line parameters in pu

For the per-unit system, please refer to RHT RE04-05 course, slides 14–17.

In [5]:
def convert_line_data_in_pu(net: pp.pandapowerNet, Z_base: float, f_base: float) -> pd.DataFrame:
    """Convert line data from per unit (p.u.) system to a Pandas DataFrame.

    Parameters
    ----------
    net : pandapowerNet
        The pandapower network for which line data needs to be converted to p.u.
    Z_base : float
        The base impedance value in ohms for the p.u. system.
    f_base : float
        The base system frequency in Hertz.

    Returns
    -------
    pd.DataFrame
        A DataFrame containing line data in per unit (p.u.) system.
    """
    lines_pu: pd.DataFrame = pd.DataFrame(
        columns=["from_bus", "to_bus", "r_pu", "x_pu", "b_pu", "z_pu", "y_pu"]
        )
    
    # Define lines' bus connections.
    lines_pu = net.line[["from_bus", "to_bus"]]
    
    # Convert line longitudinal resistance from ohm/km in p.u.
    ##YOUR CODE HERE##
    
    # Convert line longitudinal inductance from ohm/km in p.u.
    ##YOUR CODE HERE##
    
    # Convert line capacitors from nF/km to p.u.
    ##YOUR CODE HERE##
    
    # Create line longitudinal impedance and admittance.                                                    
    ##YOUR CODE HERE##
    
    # Create line transversal susceptance.
    ##YOUR CODE HERE##
    
    return lines_pu
    

### Define setpoints

In [6]:
def define_setpoints(net: pp.pandapowerNet, S_base: float)  -> tuple[np.array]:
    """Define power and voltage setpoints based on a pandapower network.

    Parameters
    ----------
    net : pandapowerNet
        The pandapower network from which setpoints are to be defined.
    S_base : float
        The system base apparent power (S_base) in MVA for power setpoints.    
    
    Returns
    -------
    tuple of numpy.ndarray
        A tuple containing the following arrays:
        - Psp (numpy.ndarray): Active power setpoints in per unit (p.u.).
        - Qsp (numpy.ndarray): Reactive power setpoints in per unit (p.u.).
        - Usp (numpy.ndarray): Voltage setpoints in complex per unit (p.u.).
    
    """
    # Define voltages starting points using external grid module and angle values
    Usp = np.array([polar2z(*net.ext_grid.loc[0, ["vm_pu", "va_degree"]].values)]*net.bus.shape[0])
    # Initialize setpoints
    Psp =  np.zeros(net.bus.shape[0])
    Qsp =  np.zeros(net.bus.shape[0])
    # Power flow algorithm use generator sign convention.
    # Loads and PV power setpoints have to be converted from MW to p.u.
    if not net.load.empty:
        # First group every load connected to the same bus and sum up their corresponding active and reactive power
        loads = pd.concat([net.bus, net.load.groupby("bus")[["p_mw", "q_mvar"]].sum()*1e6/S_base], axis=1).fillna(0)
        Psp -= loads["p_mw"].values
        Qsp -= loads["q_mvar"].values
    if not net.sgen.empty:
        # First group every generators connected to the same bus and sum up their corresponding active and reactive power
        sgen = pd.concat([net.bus, net.sgen.groupby("bus")[["p_mw", "q_mvar"]].sum()*1e6/S_base], axis=1).fillna(0)
        Psp += sgen["p_mw"].values
        Qsp += sgen["q_mvar"].values
    return Psp, Qsp, Usp

### Compute admitance matrix

For the admittance matrix theory, please refer to RHT RE04-5, slide 27.

$\left\lbrack \underline{\mathbf{I}} \right\rbrack =\left\lbrack \underline{\mathbf{Y}} \right\rbrack \cdot \left\lbrack \underline{\mathbf{U}} \right\rbrack \rightarrow \text{Current vector}_{\left(N \times 1\right)} = \text{Admitance matrix}_{\left(N \times N\right)} \cdot \text{Voltage vector}_{\left(N \times 1\right)}$

- The elements in the diagonal $\underline{Y_{ii}}$ represent the sum of all the admittances connected to the $i$-bus, with a *positive* sign.
- The elements not in the diagonal $\underline{Y_{ij}}$ correspond to the sum of every admittance directly connecting the buses together, with a *negative* sign.

The general mathematical expression follows:

$
\begin{align*}
    \underline{Y_{ij}} &= 
    \begin{cases} 
         \mathrm{j}\,b_{i} + \displaystyle\sum_l\left(\underline{y_{l}} +  \mathrm{j}\,\dfrac{b_{l}}{2}\right) &\text{if } i = j \\
        -\underline{y_{ij}} &\text{if } i \neq j
    \end{cases}
\end{align*}
$

where:
- $b_{i}$ is the sucseptance connected to the bus $i$
- $\underline{y_{l}} = \dfrac{1}{r_{l} + \mathrm{j}\,x_{l}}$ is the longitudinal admittance of lines connected to the bus $i$
- $b_{l}$ is the transversal sucseptances of lines connected to the bus $i$
- $\underline{y_{ij}}$ is the longitudinal admittance of the line connected beteween bus $i$ and $j$


**Note**: if $\left\lbrack \underline{Y} \right\rbrack$ is non-singular, we can write: $\left\lbrack \underline{\mathbf{Z}} \right\rbrack ={\left\lbrack \underline{\mathbf{Y}} \right\rbrack }^{-1}$ and we have the following matrix relationship: $\left\lbrack \underline{\mathit{\mathbf{U}}} \right\rbrack =\left\lbrack \underline{\mathit{\mathbf{Z}}} \right\rbrack \cdot \left\lbrack \underline{\mathbf{I}} \right\rbrack$. With the $\left\lbrack \underline{\mathbf{Z}} \right\rbrack$ the nodal impedance matrix.

In [7]:
def compute_admittance_matrix(line_pu: pd.DataFrame) -> np.array:
    """Compute the admittance matrix for a power system based on line data.
    
    Parameters
    ----------
    line_pu : pd.DataFrame
        A DataFrame containing line data in per unit (p.u.) system.

    Returns
    -------
    np.array
        A complex-valued numpy array representing the admittance matrix of the power system.
    """
    # Number of lines
    Nl = line_pu.shape[0]
    # Number of bus
    Nb = pd.unique(line_pu[['from_bus', 'to_bus']].values.ravel('K')).shape[0]
    # Admittance matrix initialization
    Y_bus = np.zeros([Nb, Nb], dtype=np.complex128)
    # Compute admittance matrix 
    # Non-diagonal elements
    for l in range(Nl):
        # Find bus i and j connected to line l
        ##YOUR CODE HERE##
        
        # Calculate admittance matrix non-diagonal elements i-j and j-i
        ##YOUR CODE HERE##
        
    # Diagonal elements
    for i in range(Nb):
        # Find every line connected to bus i
        ##YOUR CODE HERE##
        
        # Calculate admittance matrix diagonal element i
        ##YOUR CODE HERE##
        
    return Y_bus

### Compute power flow

For a $N$–buses grid, power flow is calculated for every bus $i$: $\underline{S_{i}} = \underline{U_{i}}\cdot\underline{I_{i}}^{*}$.

By using generalized ohm law and admittance matrix, currents are express as follows:

$
\begin{align*}
    \left\lbrack \underline{\mathbf{I}} \right\rbrack & = 
    \left\lbrack \underline{\mathbf{Y}} \right\rbrack \cdot \left\lbrack \underline{\mathbf{U}} \right\rbrack \\[0.5em]
    \begin{pmatrix} \underline{I_{1}} \\ \vdots \\ \underline{I_{N}} \end{pmatrix} & = 
    \begin{pmatrix} \underline{Y_{11}} & \cdots &  \underline{Y_{1N}} \\ \vdots & \ddots & \vdots \\ \underline{Y_{N1}} & \cdots &  \underline{Y_{NN}} \end{pmatrix} \cdot
    \begin{pmatrix} \underline{U_{1}} \\ \vdots \\ \underline{U_{N}} \end{pmatrix} \\[0.5em]
    \Rightarrow \underline{I_{i}} & = \sum_{h=1}^N\underline{Y_{ih}} \cdot \underline{U_{h}}.
\end{align*}
$

Therefore, the $i$-bus' active and reactive powers are given by:

$
\begin{align*}
    \underline{S_{i}} &= \underline{U_{i}}\cdot\left(\displaystyle\sum_{h=1}^N\underline{Y_{ih}}\cdot\underline{U_{h}}\right)^{*} = P_i + j\, Q_i \\
    & \Rightarrow 
    \begin{cases} 
    P_{i} &= \displaystyle\sum_{h=1}^N U_{i} U_{h} \cdot \lbrack G_{ih} \cos\left(\theta_{i}-\theta_{h}\right) + B_{ih} \sin\left(\theta_{i}-\theta_{h}\right)\rbrack \\
    Q_{i} &= \displaystyle\sum_{h=1}^N U_{i} U_{h} \cdot \lbrack G_{ih} \sin\left(\theta_{i}-\theta_{h}\right) - B_{ih} \cos\left(\theta_{i}-\theta_{h}\right)\rbrack
    \end{cases}
\end{align*}
$

with $G_{ih}$ and $B_{ih}$ the real and imaginary parts of the element $\underline{Y_{ih}}$ of the admittance matrix respectively.

In [8]:
def update_bus_power(Y_bus: np.array, U: np.array) -> tuple[np.array, np.array]:
    """Update the active and reactive power injections at each bus based on the admittance matrix and voltage magnitudes and angles.
    
    Parameters
    ----------
    Y_bus : np.array
        The complex-valued admittance matrix of the power system.
    U : np.array
        An array of complex-valued bus voltages.

    Returns
    -------
    tuple of np.array
        A tuple containing the following arrays:
        - P (np.array): Array of active power injections at each bus.
        - Q (np.array): Array of reactive power injections at each bus.
    """
    # Number of buses
    Nb = Y_bus.shape[0]
    # Initialize outputs
    P,  Q  = np.zeros(Nb), np.zeros(Nb)
    
    # Decompose complex voltages into polar coordinates
    ##YOUR CODE HERE##
    
    # Decompose complex admittance matrix into conductance and susceptance matrix
    ##YOUR CODE HERE##
 
    # Update bus powers 
    ##YOUR CODE HERE##
    
    return P, Q

### Build Jacobian matrix

For the Newton-Raphson theory, please refer to RHT RE04-5, slides 30–35.

The Jacobian Matrix $\mathbf{J}$ contains the partial derivatives of the equations of the injected $P$ and $Q$ with respect to the state variables. At the $k$-iteration

$
\begin{align*}
\mathbf{J}^{k}\left(\mathbf{x}\right) &= 
    \begin{pmatrix}
    \dfrac{\partial\,\mathbf{P}}{\partial\,\mathbf{\theta}}\left(\mathbf{\theta}^{k},{\mathbf{U}}^{k}\right) & \dfrac{\partial\,\mathbf{P}}{\partial\,\mathbf{U}}\left(\mathbf{\theta}^{k},{\mathbf{U}}^{k}\right) \\[1em]
    \dfrac{\partial\,\mathbf{Q}}{\partial\,\theta}\left(\mathbf{\theta}^{k},{\mathbf{U}}^{k}\right) & \dfrac{\partial\,\mathbf{Q}}{\partial\,\mathbf{U}}\left(\mathbf{\theta}^{k},{\mathbf{U}}^{k}\right)
    \end{pmatrix}\\[1em]
\text{where}: \\[1em]
\dfrac{\partial\,P_{i}}{\partial\,\theta_{j}} &=
    \begin{cases} \displaystyle\sum_{h=1;\,h \neq i}^{N} U_{i}\,U_{h}\,Y_{ih}\sin(\theta_{h}-\theta_{i}+\gamma_{ih}), \quad\quad &\text{ if } ~i=j \\
    - U_{i}\,U_{j}\,Y_{ij}\sin(\theta_{j}-\theta_{i}+\gamma_{ij}), \quad\quad &\text{ if } ~i\neq j 
    \end{cases} \\[0.5em]
\dfrac{\partial\,Q_{i}}{\partial\,\theta_{j}} &= 
    \begin{cases}\displaystyle\sum_{h=1;\,h \neq i}^{N} U_{i}\,U_{h}\,Y_{ih}\cos(\theta_{h}-\theta_{i}+\gamma_{ih}), \quad\quad &\text{ if } ~i=j \\
    -U_{i}\,U_{j}\,Y_{ij}\cos(\theta_{j}-\theta_{i}+\gamma_{ij}), \quad\quad &\text{ if } ~i\neq j 
    \end{cases}\\[2.5em]
\dfrac{\partial\,P_{i}}{\partial\,U_{j}} &= 
    \begin{cases} 2\,U_{i}\,Y_{ii}\cos(\gamma_{ii}) + \displaystyle\sum_{h=1;\,h \neq i}^{N} U_{h}\,Y_{ih}\cos(\theta_{h}-\theta_{i}+\gamma_{ih}), \quad\quad &\text{ if } ~i=j \\
    U_{i}\,Y_{ij}\sin(\theta_{j}-\theta_{i}+\gamma_{ij}), \quad\quad &\text{ if } ~i\neq j 
    \end{cases} \\[0.5em]
\dfrac{\partial\,Q_{i}}{\partial\,U_{j}} &= 
\begin{cases} - 2\,U_{i}\,Y_{ii}\sin(\gamma_{ii}) - \displaystyle\sum_{h=1;\,h \neq i}^{N} U_{h}\,Y_{ih}\sin(\theta_{h}-\theta_{i}+\gamma_{ih}), \quad\quad &\text{ if } ~i=j \\ 
-U_{i}\,Y_{ij}\sin(\theta_{j}-\theta_{i}+\gamma_{ij}), \quad\quad &\text{ if } ~i\neq j
\end{cases}
\end{align*}
$

Where $\gamma_{ih}$ is the admitance matrix $ih$-element's angle.

In [9]:
def jacobian_matrix(Y_bus: np.array, U:np.array, slack_bus: int) -> np.array:
    """Calculate the Jacobian matrix for a power system based on admittance matrix and voltage values.

    Parameters
    ----------
    Y_bus : np.array
        A complex-valued admittance matrix of the power system.
    U : np.array
        An array of complex-valued bus voltages.
    slack_bus : int
        The index of the slack bus.

    Returns
    -------
    np.array
        The Jacobian matrix of the power system, which relates changes in bus voltages to changes in active and reactive power injections.
    """
    # Number of buses
    Nb = Y_bus.shape[0]
    # Initialize active and reactive power partial differential depending on voltages modulus and angle.
    dP_dU = np.zeros([Nb, Nb])
    dP_dangle = np.zeros([Nb, Nb])
    dQ_dU = np.zeros([Nb, Nb])
    dQ_dangle = np.zeros([Nb, Nb])
    
    # Decompose complex voltages into polar coordinates
    ##YOUR CODE HERE##
    
    # Decompose complex admittance matrix into polar coordinates
    ##YOUR CODE HERE##
    
    # Calculate active and reactive power partial differential depending on voltages modulus and angle.
     ##YOUR CODE HERE##
                        
    # Remove slack nodes
    dP_dU = remove_slack_bus(dP_dU, slack_bus)
    dP_dangle = remove_slack_bus(dP_dangle, slack_bus)
    dQ_dU = remove_slack_bus(dQ_dU, slack_bus)
    dQ_dangle = remove_slack_bus(dQ_dangle, slack_bus)
    # Create Jacobian matrix
    J = np.concatenate([
            np.concatenate([dP_dU, dP_dangle], axis= 1),
            np.concatenate([dQ_dU, dQ_dangle], axis= 1)
        ])
    print("\nJaccobian Matrix:\n", pd.DataFrame(J).to_string())
    return J

### Update voltages

A N-bus grid is usually composed by:
- 1 slack bus 
- M-1 PV bus
- N-M PQ bus

Newton-Raphson methode will update voltage iteratively:

$
\begin{align*}
    \mathbf{x}^{k+1} &= \left(\mathbf{J}^{k}\right)^{-1} \cdot \mathbf{f}\left(\mathbf{x}^{k}\right) + \mathbf{x}^{k}\\
    \text{where:} & \\
    \mathbf{x}^{k} &= 
    \begin{pmatrix} 
        \theta_{2}^{k} \\ \vdots \\ \theta_{N}^{k} \\ U_{M+1}^{k} \\ \vdots \\ U_{N}^{k} 
    \end{pmatrix} \text{;} \;  
    \mathbf{f}\left(\mathbf{x}^{k}\right) = 
    \begin{pmatrix} 
        P_{2}\left(\mathbf{x}^{k}\right) - P_{2}^{SP} \\ 
        \vdots \\
        P_{N}\left(\mathbf{x}^{k}\right) - P_{N}^{SP} \\ 
        Q_{M+1}\left(\mathbf{x}^{k}\right) - Q_{M+1}^{SP} \\ 
        \vdots \\
        Q_{N}\left(\mathbf{x}^{k}\right) - Q_{N}^{SP}
    \end{pmatrix}.
\end{align*}
$

where:

- $P_{i}^{SP}$ is the bus $i$ active power setpoint
- $Q_{i}^{SP}$ is the bus $i$ reactive power setpoint


In [10]:
def update_voltages(dP: np.array, dQ: np.array, U: np.array, J: np.array, slack_bus: int) -> np.array:
    """Update the bus voltages based on changes in active and reactive power injections and the Jacobian matrix.
    
    Parameters
    ----------
    dP : np.array
        An array of changes in active power injections at each bus.
    dQ : np.array
        An array of changes in reactive power injections at each bus.
    U : np.array
        An array of complex-valued bus voltages.
    J : np.array
        The Jacobian matrix relating changes in bus voltages to changes in active and reactive power injections.
    slack_bus : int
        The index of the slack bus.

    Returns
    -------
    np.array
        An array representing the updated complex bus voltages after the power flow calculations.
    """
    # Number of buses
    Nb = U.shape[0]
    # Calculate state variable delta
    ##YOUR CODE HERE##
    
    # Update voltages in polars coordinates
    ##YOUR CODE HERE##
    
    # Build complex voltages
    U_new = ##YOUR CODE HERE##
    
    print("\nBus voltages:\n", pd.DataFrame(U_new).to_string(), "\n")
    return U_new

### Process results

Use generalized ohm law to compute lines currents ant powers flows

$
\begin{align*}
    \underline{I_{ij}} &= \left(\underline{U_{i}} - \underline{U_{j}}\right) \cdot y_{ij} + \underline{U_{i}}  \cdot b_{ij}\\
    \underline{I_{ji}} &= \left(\underline{U_{j}} - \underline{U_{i}}\right) \cdot y_{ij} + \underline{U_{j}}  \cdot b_{ij}
\end{align*}
$

In [11]:
def process_results(line_pu: pd.DataFrame, U: np.array, P: np.array, Q: np.array, V_base: float, S_base:float) -> pd.DataFrame:
    """Process power flow results and calculate line and bus-related parameters.

    Parameters
    ----------
    line_pu : pd.DataFrame
        A DataFrame containing line data in per unit (p.u.) system.
    U : np.array
        An array of complex-valued bus voltages.
    P : np.array
        An array of active power injections at each bus.
    Q : np.array
        An array of reactive power injections at each bus.
    V_base : float
        The base voltage of the system in kV.
    S_base : float
        The system base apparent power (S_base) in MVA for power setpoints.

    Returns
    -------
    pd.DataFrame, pd.DataFrame
        Two DataFrames containing the following results:
        - line_results (pd.DataFrame): Line-related parameters including active power (p_from_mw, p_to_mw), 
        reactive power (q_from_mvar, q_to_mvar), current (i_from_ka, i_to_ka) in kA.
        - bus_result (pd.DataFrame): Bus-related parameters including voltage magnitude (vm_pu), 
        voltage angle (va_degree), active power (p_mw), and reactive power (q_mvar).
    """
    # Initialize line results DataFrame
    line_results = pd.DataFrame(
        index=line_pu.index, columns=["p_from_mw", "q_from_mvar", "p_to_mw", "q_to_mvar", "i_from_ka", "i_to_ka"]
    )
    # Initialize bus results DataFrame
    bus_results = pd.DataFrame()
    # Calculate base current
    ##YOUR CODE HERE##
    
    for i in line_pu.index:
        # Find from_bus
        ##YOUR CODE HERE##
        
        # Find to_bus
        ##YOUR CODE HERE##
        
        # Find longitudinal admittance
        ##YOUR CODE HERE##
        
        # Find transversal admittance
       ##YOUR CODE HERE##
        
        # Calculate line current in pu
        i_from = ##YOUR CODE HERE##
        i_to = ##YOUR CODE HERE##
        
        # Calculate line apparent power in pu  
        s_from = ##YOUR CODE HERE##
        s_to = ##YOUR CODE HERE##
        
        # Create line results in pandapower format  
        line_results.loc[i, :]  = [
            np.real(s_from)*S_base/1e6, np.imag(s_to)*S_base/1e6, np.real(s_to)*S_base/1e6, np.imag(s_to)*S_base/1e6, 
            np.abs(i_from) * I_base/1e3, np.abs(i_to) * I_base/1e3
    # Create bus results in pandapower format     ]
    bus_results[["vm_pu", "va_degree", "p_mw", "q_mvar"]] = np.array([
        np.abs(U), np.angle(U)*180/np.pi, -P*S_base/1e6, -Q*S_base/1e6
    ]).transpose()    
    return line_results, bus_results

## Power Flow algorithm in a nutshell

1. Define bus types.
2. Define power and voltage setpoints.
3. Build admittance matrix.
4. Initialize voltages.
5. Update bus powers.
6. Calculate power mismatches: $ h\left(x^{k}\right) = \begin{bmatrix} \Delta P_{i} \left(x^{k}\right), \, \Delta Q_{i}\left(x^{k}\right) \end{bmatrix}^{T}$
7. Build Jacobian matrix.
8. Update voltage phasors in each bus.
9. Perform stop test $ \begin{cases} \max \lvert h\left(x^{k}\right) \rvert \leq \epsilon \Rightarrow &\text{the algorithm stops}. \\ \max\lvert h\left(x^{k}\right)\rvert > \epsilon \Rightarrow &\text{go to step 5}\ldots \end{cases}$


Define nominal values:

In [12]:
V_base = ##YOUR CODE HERE##
S_base = ##YOUR CODE HERE##
Z_base = ##YOUR CODE HERE##
f_base = ##YOUR CODE HERE##

Define algorithm parameters:

In [ ]:
tol_limit = 1e-4
iter_max = 10

Load data in pandapower format:

In [13]:
file_path = "5_meshed_bus.xlsx"
net = load_net_from_xlsx(file_path=file_path)

In [ ]:
# Define slack bus index
slack_bus =  ##YOUR CODE HERE##

# Define set points in pu
Psp, Qsp, Usp = define_setpoints(net = net, S_base=S_base)
# Convert line resistor and inductor into admittance complex parameter in p.u.
line_pu = convert_line_data_in_pu(net=net, Z_base=Z_base, f_base=f_base)
# Compute admittance matrix
Y_bus = compute_admittance_matrix(line_pu=line_pu)
# Initialize voltages
U=Usp
for counter in range(iter_max):
    print('Iteration {}'.format(counter + 1))
    # Power flow algorithm main steps (i.e. 5 to 8)
     ##YOUR CODE HERE##
    
    
    # Perform stop test
    tol = max(max(abs(dP)), max(abs(dQ)))
    if tol < tol_limit:
        print('Tolerance achieved: {}'.format(tol))
        print('Finished at {} iterations!'.format(counter + 1))
        break
# Compute lines and bus results
line_results, bus_results = process_results(line_pu=line_pu, U=U, P=P, Q=Q, V_base=V_base, S_base=S_base)

## Compare to **panda**power

In [15]:
pp.runpp(net)
bus_pp_results = net.res_bus[["p_mw","q_mvar", "vm_pu", "va_degree", ]]
line_pp_results = net.res_line[["p_from_mw", "q_from_mvar", "p_to_mw", "q_to_mvar", "i_from_ka", "i_to_ka"]]
line_error =(line_results-line_pp_results)/line_results
bus_error=(bus_results-bus_pp_results)/bus_results

print("Bus error:\n" + bus_error.abs().max().to_string() + "\n")
print("Line error:\n" + line_error.abs().max().to_string())

Bus error:
p_mw         1.736155e-07
q_mvar       2.615197e-07
va_degree    4.881754e-08
vm_pu        1.976358e-09

Line error:
p_from_mw      0.0
q_from_mvar    0.0
p_to_mw        0.0
q_to_mvar      0.0
i_from_ka      0.0
i_to_ka        0.0
